In [20]:
import pandas as pd
from itertools import combinations

In [22]:
nvalues = pd.read_csv('../data/nutritional_values.csv')
recipes = pd.read_csv('../data/filtered_recipes_100.csv',sep='|')

In [23]:
df = pd.merge(recipes, nvalues, on='Recipe_id', how='right')

In [29]:
filtros = ['Cena','Entrante','Plato principal','Acompañamiento']
df = df[df['Tipo'].isin(filtros)]

In [30]:
wanted_cols =['Recipe_id','Categoria','energia','grasa','proteina','carbohidratos']
df = df[wanted_cols]
cc = list(combinations(df.Recipe_id,2)) #antes de esto hacer filtradosde comida y cena etc
df.set_index('Recipe_id',inplace=True)
df

,Categoria,energia,grasa,proteina,carbohidratos
Recipe_id,,,,,
51,Recetas de Mariscos,686.782370,61.393619,17.454238,14.201002
53,Recetas de Verduras,166.307500,15.307950,2.907813,4.631500
61,Recetas de Mariscos,336.669995,5.152300,48.811874,12.279333
68,Recetas de Aperitivos y tapas,1613.028902,107.064417,50.325723,102.576002
6320,Recetas de Aves y caza,683.579012,21.922922,50.040902,69.503749
...,...,...,...,...,...
74157,Recetas de Carne,3441.640000,266.579000,182.518000,80.731200
74163,Recetas de Arroces y cereales,1482.080000,56.634200,66.497500,173.000000
74164,Recetas de Pescado,923.855879,60.862941,53.324880,40.569605


In [31]:
cc_cols = ['Recipe1','Recipe2'] #poner comida/cena whatever

combined_df = pd.DataFrame(columns = cc_cols)
combined_df['Recipe1'] = [c[0] for c in cc]
combined_df['Recipe2'] = [c[1] for c in cc]
combined_df

,Recipe1,Recipe2
0,51,53
1,51,61
2,51,68
3,51,6320
4,51,6388
...,...,...
4441685,74163,74194
4441686,74163,74200
4441687,74164,74194
4441688,74164,74200


In [33]:
macros = ['energia','grasa','proteina','carbohidratos']
for macro in macros:
    combined_df[macro] = [df[macro][c[1]]+df[macro][c[0]] for c in cc]
combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos
0,51,53,853.089870,76.701569,20.362051,18.832502
1,51,61,1023.452365,66.545919,66.266112,26.480335
2,51,68,2299.811272,168.458036,67.779961,116.777004
3,51,6320,1370.361382,83.316541,67.495140,83.704751
4,51,6388,862.652370,63.757939,20.682188,48.968202
...,...,...,...,...,...,...
4441685,74163,74194,2677.338902,118.489767,193.165848,205.539502
4441686,74163,74200,1824.970000,58.651200,99.565500,219.612000
4441687,74164,74194,2119.114781,122.718508,179.993228,73.109107
4441688,74164,74200,1266.745879,62.879941,86.392880,87.181605


In [35]:
combined_df.to_csv('../data/combined.csv', index=False)

In [9]:
goals = { #por ejemplo, estos valores sería lo que falta después de fijar desayunos etc
    'goal_energia': 2000,
    'goal_grasa': 10,
    'goal_proteina': 20,
    'goal_carbohidratos': 30
}

#func para extraer los errores
for macro in macros:
    macro_error = macro + '_error'
    macro_goal = 'goal_' + macro
    combined_df[macro_error] = abs(goals[macro_goal]-combined_df[macro])

# error total
macro_errors = [macro+'_error' for macro in macros]
combined_df['total_error'] = combined_df[macro_errors].sum(axis=1) #los sumo pero podría ser otra operación 

combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
0,1,5,138.342526,5.674834,1.590017,12.895666,1861.657474,4.325166,18.409983,17.104334,1901.496957
1,1,8,80.027500,0.425438,2.494250,10.141375,1919.972500,9.574562,17.505750,19.858625,1966.911437
2,1,10,636.090000,3.222000,30.738000,112.532000,1363.910000,6.778000,10.738000,82.532000,1463.958000
3,1,27,1693.297052,153.767161,12.876162,61.445363,306.702948,143.767161,7.123838,31.445363,489.039310
4,5,8,148.190026,5.996272,3.708267,18.573041,1851.809974,4.003728,16.291733,11.426959,1883.532394
5,5,10,704.252526,8.792834,31.952017,120.963666,1295.747474,1.207166,11.952017,90.963666,1399.870323
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267
7,8,10,645.937500,3.543438,32.856250,118.209375,1354.062500,6.456562,12.856250,88.209375,1461.584687
8,8,27,1703.144552,154.088599,14.994412,67.122738,296.855448,144.088599,5.005588,37.122738,483.072373
9,10,27,2259.207052,156.885161,43.238162,169.513363,259.207052,146.885161,23.238162,139.513363,568.843738


In [17]:
def getClosestSum(n, df):
    return df.nsmallest(n, 'total_error')

In [18]:
getClosestSum(1, combined_df)

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267


In [19]:
getClosestSum(3, combined_df) #a partir de aquí extraer los ids, meterlos en una lista para pasarlos todos

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos,energia_error,grasa_error,proteina_error,carbohidratos_error,total_error
6,5,27,1761.459578,159.337995,14.090179,69.877029,238.540422,149.337995,5.909821,39.877029,433.665267
8,8,27,1703.144552,154.088599,14.994412,67.122738,296.855448,144.088599,5.005588,37.122738,483.072373
3,1,27,1693.297052,153.767161,12.876162,61.445363,306.702948,143.767161,7.123838,31.445363,489.039310
